## Desarrollo de funciones para la Api

+ def **PlayTimeGenre( *`genero` : str* )**:
    Debe devolver `año` con mas horas jugadas para dicho género.
  
    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

+ def **UsersRecommend( *`año` : int* )**:
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

+ def **UsersNotRecommend( *`año` : int* )**:
   Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
  
   Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

+ def **sentiment_analysis( *`año` : int* )**:
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. 

    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}


In [20]:
#Importar librerias
import pandas as pd
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq

### 1. Función **def PlayTimeGenre**(genero:str):


Requiero de las tablas steam_games que contiene el genero y año de lanzamiento. Por otro lado la tabla
user_items que contiene la variable playtimeforever, que sería horas/minutos judados

In [3]:
#Traigo la tabla steam games
steam_games= pd.read_csv('data/steam_7_api.csv', encoding='utf8')
steam_games.head(2)

,item_id,app_name,title,genres,Año de lanzamiento
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Action,2018
1,761140,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018


In [29]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
genre= steam_games[["item_id","genres","Año de lanzamiento"]]

In [5]:
#Traigo la tabla user_items para unirlas
user_items= pd.read_csv('data/items_3.csv', encoding='utf8')
user_items.head(2) 

,item_id,item_name,playtime_forever,user_id,steam_id,items_count
0,10,Counter-Strike,6,76561197970982479,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,76561197970982479,277


In [6]:
#me quedo solo con la variable que necesito
play_time= user_items[["item_id","playtime_forever"]]
play_time.head(3)

,item_id,playtime_forever
0,10,6
1,20,0
2,30,7


In [30]:
#unifico
genre_2= play_time.merge(genre, on="item_id")
genre_2.head(5)

,item_id,playtime_forever,genres,Año de lanzamiento
0,10,6,Action,2000
1,10,0,Action,2000
2,10,0,Action,2000
3,10,93,Action,2000
4,10,108,Action,2000


Entonces ahora debo agrupar por genero y año las horas jugadas
+ Año de lanzamiento con más horas jugadas para Género X

In [31]:
horas_anio_genre=genre_2.groupby(["genres","Año de lanzamiento"]).playtime_forever.sum().reset_index(name='Horas jugadas')
horas_anio_genre.sort_values(by="Horas jugadas", ascending=False)

,genres,Año de lanzamiento,Horas jugadas
27,Action,2012,1099062852
163,Indie,2006,452672790
265,Simulation,2006,451105400
28,Action,2013,420745592
26,Action,2011,335808266
...,...,...,...
127,Education,2010,9
247,Simulation,1988,1
196,RPG,1991,0
279,Software Training,2010,0


+ Año de lanzamiento con más horas jugadas para Género X, ademas el formato string para genero

In [32]:
horas_anio_genre['genres']=horas_anio_genre['genres'].astype('string') # pasar de objeto a string
horas_anio_genre['Año de lanzamiento']=horas_anio_genre['Año de lanzamiento'].astype('string')

 debemos encontrar el maximo de horas jugadas por genero

In [33]:
res = horas_anio_genre.groupby("genres")["Horas jugadas"].agg(Horas="max").reset_index()
res.head()


,genres,Horas
0,Action,1099062852
1,Adventure,224347981
2,Animation &amp; Modeling,1354685
3,Audio Production,466246
4,Casual,82515351


In [34]:
res.shape

(21, 2)

In [35]:
#para realizar el join, todas las variables de deben llamar iguales, asi cuando haga el merge, me buscara solamente la info que
#necesito para completar mi tabla res, que contiene las maximas horas por genero
genero2 = horas_anio_genre.rename(columns={'Horas jugadas':'Horas'})
genero2


,genres,Año de lanzamiento,Horas
0,Action,0,4770730
1,Action,1983,3582
2,Action,1984,384
3,Action,1988,16243
4,Action,1989,607
...,...,...,...
349,Web Publishing,2013,335849
350,Web Publishing,2014,33732
351,Web Publishing,2015,348861
352,Web Publishing,2016,136


In [36]:
tabla_final =pd.merge(res,genero2)
tabla_final

,genres,Horas,Año de lanzamiento
0,Action,1099062852,2012
1,Adventure,224347981,2011
2,Animation &amp; Modeling,1354685,2015
3,Audio Production,466246,2014
4,Casual,82515351,2015
5,Design &amp; Illustration,1947406,2012
6,Early Access,120366296,2013
7,Education,342804,2013
8,Free to Play,147728688,2013
9,Indie,452672790,2006


In [37]:
def PlayTimeGenre(genero):
    anio= tabla_final[tabla_final["genres"]==  genero ]["Año de lanzamiento"].iloc[0]
    horas_jugadas= tabla_final[tabla_final["genres"]== genero]["Horas"].iloc[0]
    return {"El género": genero, "en el año de lanzamiento": int(anio),"tiene más horas jugadas": int(horas_jugadas)}

In [38]:
PlayTimeGenre('Action')

{'El género': 'Action',
 'en el año de lanzamiento': 2012,
 'tiene más horas jugadas': 1099062852}

In [124]:
#se guarda para la api
tabla_final2 = "funcion_1.csv"
tabla_final.to_csv(tabla_final2, index=False, encoding="utf-8")

In [125]:
#convertir a parquet las funciones

#Transformo el archivo csv a parquet
#Leo el archivo csv
funcion_1= pd.read_csv("funcion_1.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_1.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_1)
pq.write_table(table,output_file)

### 2. Función **def UserForGenre(genero:str)**:

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [23]:
#Sigo con la misma tabla de steams + items, pero ahora incluyo al user_id
#el user_id esta como object, lo tengo que pasar a string, asi que medio que debo a rehacer casi todo lo de la funcion 1
user_items.head(2)

,item_id,item_name,playtime_forever,user_id,steam_id,items_count
0,10,Counter-Strike,6,76561197970982479,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,76561197970982479,277


In [24]:
user_items2=user_items.copy() #creo una tabla extra por las dudas

In [25]:
user_items2['user_id']=user_items2['user_id'].astype('string') #paso de object a string

In [26]:
play_time_user= user_items[["item_id","playtime_forever","user_id"]] # me quedo con las variables necesarias
play_time_user.head(3)

,item_id,playtime_forever,user_id
0,10,6,76561197970982479
1,20,0,76561197970982479
2,30,7,76561197970982479


In [40]:
genre.head(3) #verifico antes mi tabla a curzar

,item_id,genres,Año de lanzamiento
0,761140,Action,2018
1,761140,Casual,2018
2,761140,Indie,2018


In [41]:
#unifico
genre_user= play_time_user.merge(genre, on="item_id")
genre_user.head(5)

,item_id,playtime_forever,user_id,genres,Año de lanzamiento
0,10,6,76561197970982479,Action,2000
1,10,0,js41637,Action,2000
2,10,0,Riot-Punch,Action,2000
3,10,93,doctr,Action,2000
4,10,108,corrupted_soul,Action,2000


In [42]:
genre_user['genres']=genre_user['genres'].astype('string') #pasar de object a string
genre_user['Año de lanzamiento']=genre_user['Año de lanzamiento'].astype('string')
genre_user['user_id']=genre_user['user_id'].astype('string')

In [43]:
#agrupo el genero por usuario y horas jugadas
user_hours_year= genre_user.groupby(["genres","user_id","Año de lanzamiento"])["playtime_forever"].sum().reset_index()
user_hours_year

,genres,user_id,Año de lanzamiento,playtime_forever
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363
...,...,...,...,...
3449993,Web Publishing,zepavil,2015,9010
3449994,Web Publishing,zeshirky,2007,1
3449995,Web Publishing,zevlupine,2012,4
3449996,Web Publishing,zilaman,2013,9


In [44]:
#necesito saber los usuarios con mayor horas jugadas por genero
user_hours_total= genre_user.groupby(["genres","user_id"])["playtime_forever"].sum().reset_index()
user_hours_total.head(3)

,genres,user_id,playtime_forever
0,Action,--000--,139469
1,Action,--ace--,69325
2,Action,--ionex--,38315


In [45]:
#por cada genero el total de horas sumadas
max_user = user_hours_total.groupby(["genres"])["playtime_forever"].agg(playtime_forever="max").reset_index()
max_user

,genres,playtime_forever
0,Action,1699307
1,Adventure,2191551
2,Animation &amp; Modeling,168314
3,Audio Production,109916
4,Casual,1224933
5,Design &amp; Illustration,168314
6,Early Access,316969
7,Education,65427
8,Free to Play,808241
9,Indie,2402994


In [46]:
tabla_user =pd.merge(max_user,user_hours_total)
tabla_user #ya tengo el usuario con mayor horas jugadas por genero, ahora me falta incluir el total por año

,genres,playtime_forever,user_id
0,Action,1699307,Sp3ctre
1,Adventure,2191551,REBAS_AS_F-T
2,Animation &amp; Modeling,168314,ScottyG555
3,Audio Production,109916,Lickidactyl
4,Casual,1224933,REBAS_AS_F-T
5,Design &amp; Illustration,168314,ScottyG555
6,Early Access,316969,76561197978756659
7,Education,65427,76561198059330972
8,Free to Play,808241,idonothack
9,Indie,2402994,REBAS_AS_F-T


In [47]:
user_hours_year2=user_hours_year.rename(columns={'playtime_forever':'playtime_forever_year'})
user_hours_year2.head(5)

,genres,user_id,Año de lanzamiento,playtime_forever_year
0,Action,--000--,2009,5329
1,Action,--000--,2010,22
2,Action,--000--,2011,6522
3,Action,--000--,2012,109346
4,Action,--000--,2013,363


In [48]:
#voy a probar si el cruce funciona ok, en el caso del usuario REBAS_AS_F-T	
prueba3=user_hours_year2[(user_hours_year2['user_id'] == 'REBAS_AS_F-T') & (user_hours_year2['genres']=='Indie')]
prueba3
#Para el usuario REBAS_AS_F-T la tabla tabla_user me debe traer 16 registros

,genres,user_id,Año de lanzamiento,playtime_forever_year
1970474,Indie,REBAS_AS_F-T,1999,0
1970475,Indie,REBAS_AS_F-T,2001,11
1970476,Indie,REBAS_AS_F-T,2003,1863
1970477,Indie,REBAS_AS_F-T,2005,0
1970478,Indie,REBAS_AS_F-T,2006,1673
1970479,Indie,REBAS_AS_F-T,2007,1070
1970480,Indie,REBAS_AS_F-T,2008,1366
1970481,Indie,REBAS_AS_F-T,2009,28993
1970482,Indie,REBAS_AS_F-T,2010,21487
1970483,Indie,REBAS_AS_F-T,2011,100155


In [49]:
tabla_user2 =pd.merge(tabla_user,user_hours_year2)
tabla_user2

,genres,playtime_forever,user_id,Año de lanzamiento,playtime_forever_year
0,Action,1699307,Sp3ctre,0,1657
1,Action,1699307,Sp3ctre,1993,0
2,Action,1699307,Sp3ctre,1995,217
3,Action,1699307,Sp3ctre,1996,0
4,Action,1699307,Sp3ctre,1998,0
...,...,...,...,...,...
165,Utilities,207651,76561198073642113,2014,207651
166,Video Production,168314,ScottyG555,2015,168314
167,Web Publishing,142964,Xyphien,2005,7296
168,Web Publishing,142964,Xyphien,2012,64657


In [50]:
#verificamos
prueba4=tabla_user2[(tabla_user2['user_id'] == 'REBAS_AS_F-T') & (tabla_user2['genres']=='Indie')]
prueba4.shape # esta ok
#ya tenemos la tabla para crear la funcion

(16, 5)

In [56]:
tabla_user2 = tabla_user2.rename(columns={'Año de lanzamiento':'Año', 'playtime_forever_year':'Horas jugadas'})
tabla_user2["Año"] = pd.to_numeric(tabla_user2["Año"])

In [57]:
def UserForGenre(genero):
    usuario= tabla_user2[tabla_user2["genres"]== genero]["user_id"].iloc[0] #obtengo usuario
    historial=tabla_user2[(tabla_user2['user_id'] == usuario) & (tabla_user2['genres']==genero)] #filtro por el genero y usuario
    historial2 = historial[['Año', 'Horas jugadas']].copy() #me quedo con las columnas necesarias
    historial3=historial2.to_dict(orient="records")
    return {"Usuario":usuario ,"con más horas jugadas para": genero, "Historial acumulado": historial3 }

In [58]:
UserForGenre('Indie')

{'Usuario': 'REBAS_AS_F-T',
 'con más horas jugadas para': 'Indie',
 'Historial acumulado': [{'Año': 1999, 'Horas jugadas': 0},
  {'Año': 2001, 'Horas jugadas': 11},
  {'Año': 2003, 'Horas jugadas': 1863},
  {'Año': 2005, 'Horas jugadas': 0},
  {'Año': 2006, 'Horas jugadas': 1673},
  {'Año': 2007, 'Horas jugadas': 1070},
  {'Año': 2008, 'Horas jugadas': 1366},
  {'Año': 2009, 'Horas jugadas': 28993},
  {'Año': 2010, 'Horas jugadas': 21487},
  {'Año': 2011, 'Horas jugadas': 100155},
  {'Año': 2012, 'Horas jugadas': 148459},
  {'Año': 2013, 'Horas jugadas': 169349},
  {'Año': 2014, 'Horas jugadas': 326927},
  {'Año': 2015, 'Horas jugadas': 751765},
  {'Año': 2016, 'Horas jugadas': 815989},
  {'Año': 2017, 'Horas jugadas': 33887}]}

In [126]:
#se guarda para la api
tabla_user3 = "funcion_2.csv"
tabla_user2.to_csv(tabla_user3, index=False, encoding="utf-8")

In [127]:
#guardo parquet
funcion_2= pd.read_csv("funcion_2.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_2.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_2)
pq.write_table(table,output_file)

### 3. Funcion **defUsersRecommend(año:int)**:

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Solo uso la tabla de reviews, ya que seria la recomendación del año dado, y la misma contiene posted, el momento que se publico
Utilizar el año de lanzamiento mezclando con comentarios de otro año, podria no ser consistente.
De todas maneras steams, te dice el nombre

In [72]:
games_users= steam_games[["item_id","app_name","Año de lanzamiento"]]

In [73]:
#saco los duplicados por genero, ya que en este caso no lo utilizaremos
games_users=game_users.drop_duplicates()

In [74]:
games_users.shape

(32132, 3)

In [75]:
games_users.dtypes

item_id                int64
app_name              object
Año de lanzamiento     int64
dtype: object

In [67]:
reviews= pd.read_csv('data/reviews_9.csv', encoding='utf8')
reviews.head(2)

,user_id,item_id,helpful,recommend,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,2011,2,0.337500


In [69]:
reviews2= reviews[["user_id","item_id","recommend","year"]] # me quedo solo con las variables necesarias

In [76]:
#unifico
tabla_reviews= games_users.merge(reviews2, on="item_id", how="right")
tabla_reviews.shape

(59305, 6)

In [77]:
tabla_reviews=tabla_reviews.drop_duplicates()
tabla_reviews.shape #vs 59305, ahora hay menos casos

(58852, 6)

In [78]:
#convierto la tabla recommend de boleando a 1/0
tabla_reviews["recommend"] = tabla_reviews["recommend"].astype(int)
tabla_reviews.head(5)

,item_id,app_name,Año de lanzamiento,user_id,recommend,year
0,1250,Killing Floor,2009.0,76561197970982479,1,2011
1,22200,Zeno Clash,2009.0,76561197970982479,1,2011
2,43110,NaN,NaN,76561197970982479,1,2011
3,251610,NaN,NaN,js41637,1,2014
4,227300,Euro Truck Simulator 2,2013.0,js41637,1,2013


In [79]:
top_reviews=tabla_reviews.groupby(["item_id","app_name","year"]).recommend.sum().reset_index()
top_reviews

,item_id,app_name,year,recommend
0,10,Counter-Strike,0,6
1,10,Counter-Strike,2011,1
2,10,Counter-Strike,2012,3
3,10,Counter-Strike,2013,7
4,10,Counter-Strike,2014,22
...,...,...,...,...
5992,521340,True or False,0,2
5993,521430,Super Switch,0,1
5994,521570,You Have 10 Seconds 2,0,2
5995,521990,Galactic Storm,0,1


In [80]:
max_reviews = top_reviews.groupby(["year"])["recommend"].nlargest(3).reset_index()
max_reviews

,year,level_1,recommend
0,0,125,633
1,0,86,245
2,0,260,147
3,2010,87,10
4,2010,133,6
5,2010,119,4
6,2011,88,79
7,2011,113,26
8,2011,1722,25
9,2012,89,267


In [81]:
max_reviews2=pd.merge(max_reviews,top_reviews)
max_reviews2

,year,level_1,recommend,item_id,app_name
0,0,125,633,730,Counter-Strike: Global Offensive
1,0,86,245,440,Team Fortress 2
2,0,260,147,4000,Garry's Mod
3,2010,87,10,440,Team Fortress 2
4,2010,133,6,1250,Killing Floor
5,2010,119,4,630,Alien Swarm
6,2011,88,79,440,Team Fortress 2
7,2011,113,26,620,Portal 2
8,2011,1722,25,105600,Terraria
9,2012,89,267,440,Team Fortress 2


In [82]:
#ahora necesito ordenarlo en el top 3, y agregar puesto 1, 2, 3 
max_reviews2.sort_values(by = ['year', 'recommend'], ascending=False)
max_reviews2['Acumulado'] = max_reviews2.groupby([max_reviews2['year']])['year'].cumcount()
max_reviews2['Acumulado']=max_reviews2['Acumulado'].replace([0,1,2],['Puesto 1','Puesto 2','Puesto 3'])
max_reviews3=max_reviews2.drop(columns=['level_1'])
max_reviews3

,year,recommend,item_id,app_name,Acumulado
0,0,633,730,Counter-Strike: Global Offensive,Puesto 1
1,0,245,440,Team Fortress 2,Puesto 2
2,0,147,4000,Garry's Mod,Puesto 3
3,2010,10,440,Team Fortress 2,Puesto 1
4,2010,6,1250,Killing Floor,Puesto 2
5,2010,4,630,Alien Swarm,Puesto 3
6,2011,79,440,Team Fortress 2,Puesto 1
7,2011,26,620,Portal 2,Puesto 2
8,2011,25,105600,Terraria,Puesto 3
9,2012,267,440,Team Fortress 2,Puesto 1


In [85]:
#volvemos a string el año , porque la api lo toma como string y no me deja poner int
max_reviews3['year']=max_reviews3['year'].apply(str)

In [86]:
def UsersRecommend(anio:int): #el int hay que poner en la api para que funcione, este codigo corrio
    tabla1=max_reviews3[max_reviews3['year'] == anio]
    tabla1.reset_index()
    
    dato = tabla1[tabla1["year"]== anio]["app_name"].iloc[0]
    dato1 = tabla1[tabla1["year"]== anio]["app_name"].iloc[1]
    dato2 = tabla1[tabla1["year"]== anio]["app_name"].iloc[2]
    
    return {"Los juegos más recomendados para el año": año, "Puesto 1": dato,"Puesto 2": dato1,"Puesto 3": dato2}

In [89]:
def UsersRecommend(anio): #este  codigo no corrio en la api
    anio=str(anio)
    tabla1=max_reviews3[max_reviews3['year'] == anio]
    tabla1.reset_index()
    
    dato = tabla1[tabla1["year"]== anio]["app_name"].iloc[0]
    dato1 = tabla1[tabla1["year"]== anio]["app_name"].iloc[1]
    dato2 = tabla1[tabla1["year"]== anio]["app_name"].iloc[2]
    
    return {"Los juegos más recomendados para el año": anio, "Puesto 1": dato,"Puesto 2": dato1,"Puesto 3": dato2}

In [90]:
UsersRecommend(2014)

{'Los juegos más recomendados para el año': '2014',
 'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Counter-Strike: Global Offensive',
 'Puesto 3': "Garry's Mod"}

In [128]:
#se guarda para la api
max_reviews4 = "funcion_3.csv"
max_reviews3.to_csv(max_reviews4, index=False, encoding="utf-8")

In [129]:
funcion_3= pd.read_csv("funcion_3.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_3.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_3)
pq.write_table(table,output_file)

### 4. Funcion **def UsersNotRecommend( año:int )**:

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [94]:
#Parto de la funcion anterior, en vez del max, el min
min_reviews = top_reviews.groupby(["year"])["recommend"].nsmallest(3).reset_index()
min_reviews.head(2)

,year,level_1,recommend
0,0,205,0
1,0,382,0


In [95]:
min_reviews2=pd.merge(min_reviews,top_reviews)
min_reviews2.head(5) #hay muchos juegos sin recomendaciones, pero elegimos los primeros 3 que aparezcan en la tabla

,year,level_1,recommend,item_id,app_name
0,0,205,0,2850,X: Tension
1,0,205,0,7830,Men of War™
2,0,205,0,8140,Tomb Raider: Underworld
3,0,205,0,8170,Battlestations Pacific
4,0,205,0,10220,Postal III


In [96]:
min_reviews2.sort_values(by = ['year', 'recommend'], ascending=False)
min_reviews2=min_reviews2.drop(columns=['level_1'])
min_reviews2['year']=min_reviews2['year'].astype('string')

In [97]:
def UsersNotRecommend(año):
    año=str(año)
    tabla11=min_reviews2[min_reviews2['year']==año]
    tabla22=tabla11[['app_name','year']].copy() #me quedo con las columnas necesarias
    tabla22.reset_index()
    
    dato0 = tabla22[tabla22["year"]==año]["app_name"].iloc[0]
    dato11 = tabla22[tabla22["year"]== año]["app_name"].iloc[1]
    dato22 = tabla22[tabla22["year"]==año]["app_name"].iloc[2]
    
    return {"Los juegos menos recomendados para el año": año, "Puesto 1": dato0,"Puesto 2": dato11,"Puesto 3": dato22}

In [98]:
UsersNotRecommend(2014)

{'Los juegos menos recomendados para el año': '2014',
 'Puesto 1': 'Half-Life Deathmatch: Source',
 'Puesto 2': 'Earth 2160',
 'Puesto 3': 'Vigil: Blood Bitterness™'}

In [130]:
#se guarda para la api
min_reviews3 = "funcion_4.csv"
min_reviews2.to_csv(min_reviews3, index=False, encoding="utf-8")

In [131]:
funcion_4= pd.read_csv("funcion_4.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_4.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_4)
pq.write_table(table,output_file)

### 5. Función **def sentiment_analysis(año:int)**:


Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. 

    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [115]:
games_users.head() #reviso la tabla a utilizar

,item_id,app_name,Año de lanzamiento
0,761140,Lost Summoner Kitty,2018
5,643980,Ironbound,2018
9,670290,Real Pool 3D - Poolians,2017
14,767400,弹炸人2222,2017
17,773570,Log Challenge,0


In [116]:
games_users=game_users.drop_duplicates()

In [117]:
games_users.shape

(32132, 3)

In [118]:
reviews.head(2)

,user_id,item_id,helpful,recommend,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,2011,2,0.337500


In [119]:
sentimiento_analysis=reviews.merge(games_users, on="item_id")
sentimiento_analysis.head(3)

,user_id,item_id,helpful,recommend,year,sentiment_analisis,polaridad,app_name,Año de lanzamiento
0,76561197970982479,1250,No ratings yet,True,2011,1,0.174444,Killing Floor,2009
1,death-hunter,1250,No ratings yet,True,2015,1,0.204167,Killing Floor,2009
2,DJKamBer,1250,No ratings yet,True,2013,1,0.110417,Killing Floor,2009


In [120]:
sentimiento_analysis.shape

(53988, 9)

In [121]:
def sentiment_analysis(anio):
    
    reviews_por_anio=sentimiento_analysis[sentimiento_analysis["Año de lanzamiento"]== anio]
    
 
    Negativos = 0
    Neutral = 0
    Positivos = 0
    

    for i in reviews_por_anio["sentiment_analisis"]:
        if i == 0:
            Negativos += 1
        elif i == 1:
            Neutral += 1 
        elif i == 2:
            Positivos += 1

    count_sentiment ={"Negative": Negativos , "Neutral" : Neutral, "Positive": Positivos}
    
    return count_sentiment

In [122]:
sentiment_analysis(2015)

{'Negative': 437, 'Neutral': 4794, 'Positive': 1504}

In [132]:
sentimiento_analysis2 = "funcion_5.csv"
sentimiento_analysis.to_csv(sentimiento_analysis2, index=False, encoding="utf-8")

In [133]:
funcion_5= pd.read_csv("funcion_5.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/funcion_5.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(funcion_5)
pq.write_table(table,output_file)